In [ ]:
#recommend books based on poularity and content-based

In [8]:
import warnings
warnings.filterwarnings('ignore')
import os 
import pandas as pd
import numpy as np

In [9]:
file= 'books.csv'
# Load the dataset
df = pd.read_csv(file, on_bad_lines='skip')

In [10]:
df.describe()

,bookID,average_rating,isbn13,num_pages,ratings_count,text_reviews_count
count,11123.000000,11123.000000,1.112300e+04,11123.000000,1.112300e+04,11123.000000
mean,21310.856963,3.934075,9.759880e+12,336.405556,1.794285e+04,542.048099
std,13094.727252,0.350485,4.429758e+11,241.152626,1.124992e+05,2576.619589
min,1.000000,0.000000,8.987060e+09,0.000000,0.000000e+00,0.000000
25%,10277.500000,3.770000,9.780345e+12,192.000000,1.040000e+02,9.000000
50%,20287.000000,3.960000,9.780582e+12,299.000000,7.450000e+02,47.000000
75%,32104.500000,4.140000,9.780872e+12,416.000000,5.000500e+03,238.000000
max,45641.000000,5.000000,9.790008e+12,6576.000000,4.597666e+06,94265.000000


In [ ]:
def popularity_recommender(df, min_percentile=0.9, top_n=10):
    """
    Returns top N books recommended based on popularity using a weighted IMDb-style rating.

    Parameters:
    - df: DataFrame containing 'title', 'average_rating', and 'ratings_count'
    - min_percentile: minimum ratings threshold percentile to consider
    - top_n: number of books to recommend
    """

    # Filter only books with sufficient number of votes
    C = df['average_rating'].mean()
    m = df['ratings_count'].quantile(min_percentile)

    qualified = df[df['ratings_count'] >= m].copy()

    # Calculate the weighted rating
    def weighted_rating(x, m=m, C=C):
        v = x['ratings_count']
        R = x['average_rating']
        return (v / (v + m)) * R + (m / (v + m)) * C

    qualified['score'] = qualified.apply(weighted_rating, axis=1)

    # Sort and return top N
    return qualified.sort_values('score', ascending=False)[['title', 'average_rating', 'ratings_count', 'score']].head(top_n)


In [11]:
def popularity_recommender(df, percentile = 0.9, top_n = 10):
    C = df['average_rating'].mean()
    m = df['average_rating'].quantile(percentile)
 
    #filter the qualifying books
    qualified = df[df['ratings_count'] >= m].copy()
 
    #compute weighted rating
    def weighted_rating(x, m=m, C=C):
        v = x['ratings_count']
        R = x['average_rating']
        return (v/(v+m) * R) + (m/(m+v) * C)
 
    qualified['score'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('score', ascending=False)
 
    return qualified[['title','authors','average_rating','ratings_count','score']].head(top_n)
 
top_books = popularity_recommender(df, percentile = 0.9, top_n = 10)
print(top_books)

                                                   title  \
6587                      The Complete Calvin and Hobbes   
4      Harry Potter Boxed Set  Books 1-5 (Harry Potte...   
6589       It's a Magical World (Calvin and Hobbes  #11)   
6           Harry Potter Collection (Harry Potter  #1-6)   
6590   Homicidal Psycho Jungle Cat (Calvin and Hobbes...   
6593   Calvin and Hobbes: Sunday Pages 1985-1995: An ...   
1848                                         Early Color   
5614                                    Study Bible: NIV   
6497      The Complete Aubrey/Maturin Novels (5 Volumes)   
10937  The Price of the Ticket: Collected Nonfiction ...   

                           authors  average_rating  ratings_count     score  
6587                Bill Watterson            4.82          32213  4.819882  
4       J.K. Rowling/Mary GrandPré            4.78          41428  4.779912  
6589                Bill Watterson            4.76          23875  4.759852  
6                     J.K. 